In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from matplotlib import cm
import sklearn.linear_model

In [2]:
x = np.array([[-0.86, -0.30, -0.05, 0.73]])
n = np.array([[5., 5., 5., 5.]])
y = np.array([[0, 1, 3, 5]])

Probabilistic model:

\begin{align*}
y_i | \gamma_i &\sim  \mathrm{Bin}(n_i, \gamma_i)\\
\rm{logit}(\gamma_i) &= \alpha + \beta x_i\\
\theta = 
\begin{bmatrix}
\alpha \\
\beta
\end{bmatrix} &\sim \mathcal{N}(\mu_0, \Sigma_0), \qquad \text{where } 
\mu_0 = \begin{bmatrix}
0 \\
10
\end{bmatrix}, \;\;
\Sigma_0 = \begin{bmatrix}
4 & 12 \\
12 & 100
\end{bmatrix}
\end{align*}

where

\begin{align*}
\rm{logit}(z) &= \log \left( \frac{p}{1-p} \right),\\
\rm{logit}^{-1}(p) &= \rm{sigm}(p) = \frac{1}{1 + e^{-p}}
\end{align*}

Let us first obtain a deterministic estimate of $\alpha$ and $\beta$ by solving the maximum likelihood problem:

$$\alpha^{\rm ml}, \beta^{\rm ml} = \arg \min_{\alpha, \beta} \mathrm{Bin}(n_i, \rm{sigm}(\alpha + \beta x_i))$$ 

In [ ]:
# Get an initial deterministic estimate of alpha and beta (logistic regression)
#logreg = sklearn.linear_model.LogisticRegression()
#logreg.fit(x.transpose(), 1/5*y.reshape(-1)) # does not work with continuous labels

In [ ]:
# sigmoid function, equivalent to logit^{-1}
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [ ]:
# Likelihood of the 4 observations (neglecting a constant multiplicative factor).
# The overall likelihood is the product of all terms.
lik_fun = lambda alpha, beta: sigmoid(alpha+beta*x)**y * (1 - sigmoid(alpha+beta*x))**(n-y)

Let us visualize the likelihood function in 2D

In [ ]:
dalpha = 0.01
dbeta = 0.01
ALPHA = np.arange(-4, 10, dalpha)
BETA = np.arange(-10, 40, dbeta)

AA, BB = np.meshgrid(ALPHA, BETA, indexing='xy')
LL = lik_fun(AA[..., np.newaxis], BB[..., np.newaxis])
LL = np.prod(LL, axis=-1)

In [ ]:
fig, ax = plt.subplots()
c = ax.pcolormesh(AA, BB, LL, cmap=cm.coolwarm, shading='auto')
fig.colorbar(c, ax=ax)
ax.set_title(f"Likelihood");
ax.set_xlabel(r"$\alpha$");
ax.set_ylabel(r"$\beta$");

In [ ]:
plt.contour(AA, BB, LL); #, levels=[5, 15,  95]); # levels=[5, 15, 25, 35, 45, 55, 65, 75, 85, 95])
plt.xlabel(r"$\alpha$");
plt.ylabel(r"$\beta$");
plt.grid(True)

Let us introduce a prior on the parameters $\alpha$ and $\beta$:
\begin{equation}
\begin{bmatrix}
\alpha \\
\beta
\end{bmatrix} \sim \mathcal{N}(\mu_0, \Sigma_0), \qquad \text{where } 
\mu_0 = \begin{bmatrix}
0 \\
10
\end{bmatrix}, \;\;
\Sigma_0 = \begin{bmatrix}
4 & 12 \\
12 & 100
\end{bmatrix}
\end{equation}

Now we can go for a full Bayesian estimation of $\alpha$ and $\beta$:

\begin{equation}
p(\alpha, \beta) = \frac{p(y | \alpha, \beta) p(\alpha, \beta)}{p(y)}
\end{equation}

There is no closed-form solution in our case, but we still have two options:

1. Obtain $p(y | \alpha, \beta) p(\alpha, \beta)$ and normalize it to a valid distribution
2. Use MCMC to get samples...

In [ ]:
mu = np.array([0, 10])
cov_0 = np.array([[4, 12], [12, 100]]) 
prior_fun = stats.multivariate_normal(mean=mu, cov=cov_0)

In [ ]:
dalpha = 0.01
dbeta = 0.01
ALPHA = np.arange(-4, 10, dalpha)
BETA = np.arange(-10, 40, dbeta)

AA, BB = np.meshgrid(ALPHA, BETA, indexing='xy')
LL = lik_fun(AA[..., np.newaxis], BB[..., np.newaxis])
LL = np.prod(LL, axis=-1) # Likelihood

AABB = np.stack((AA, BB), axis=-1)
PP = prior_fun.pdf(AABB) # Prior

POST_UNSC = LL * PP
normalizing_factor = np.sum(POST_UNSC)*dalpha*dbeta
POST_SC = POST_UNSC/normalizing_factor

In [ ]:
fig, ax = plt.subplots()
c = ax.pcolormesh(AA, BB, POST_SC, cmap=cm.coolwarm, shading='auto')
fig.colorbar(c, ax=ax)
ax.set_title(f"Posterior distribution");
ax.set_xlabel(r"$\alpha$");
ax.set_ylabel(r"$\beta$");

In [ ]:
In this case the 